In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import zipfile

# Unzip the archive
with zipfile.ZipFile('C:/Users/sarav/FakeNewsDetector/FakeNewsDetector/.ipynb_checkpoints/fake_or_real_news.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('C:/Users/sarav/FakeNewsDetector/FakeNewsDetector/.ipynb_checkpoints/')

# Load the dataset
fake_and_real_news_df = pd.read_csv('C:/Users/sarav/FakeNewsDetector/FakeNewsDetector/.ipynb_checkpoints/fake_or_real_news.csv')

# Function to clean text
def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

# Apply cleaning function
fake_and_real_news_df['text'] = fake_and_real_news_df['text'].apply(clean_text)

# Split data
X = fake_and_real_news_df['text']
y = fake_and_real_news_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'), max_df=0.7)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize the model
model = LogisticRegression()

# Train the model
model.fit(X_train_tfidf, y_train)

# Predict on the test data
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Classification Report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

# Save the model
import joblib
joblib.dump(model, 'fake_news_detector_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

# Load the model and vectorizer
model = joblib.load('fake_news_detector_model.pkl')
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Predict function
def predict_fake_news(text):
    text = clean_text(text)
    text_tfidf = tfidf_vectorizer.transform([text])
    prediction = model.predict(text_tfidf)
    return prediction[0]

# Example usage
new_article = "Trump slams 'corrupt' media, as more women make sexual assault claims."
print(f"The article is: {'Fake' if predict_fake_news(new_article) == 1 else 'Real'}")


ModuleNotFoundError: No module named 'pandas'